In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTSwimmer import GPTSwimmerEnv

In [2]:

folder_name = "results/Eureka_morphology"
log_file = os.path.join(folder_name, "parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

iterations = 5
morphology_nums = 16
rewardfunc_nums = 1

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []


In [ ]:
import prompts
class DGA:
    def __init__(self):
        api_key = "<api_key>"
        self.client = OpenAI(api_key=api_key)
        self.model = "gpt-4o-mini"


    def generate_morphology_eureka(self, morphology_nums, best_message, folder_name):

        messages = [
                    {"role": "system", "content": "You are a helpful mujoco robot designer"},
                    {"role": "user", "content": morphology_prompts + best_message + morphology_format}
                    ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums
        )

        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_swimmer_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = swimmer_design(parameter)  
            filename = f"GPTSwimmer_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
        


In [4]:
eureka_morphology_prompts = """
Role: You are a robot designer trying to design robot parameters to increase the fitness function as effective as possible.
Task: Your task is to design parameters of robot that will help agent achieve the fitness function as high as possible.
fintess function: walk distance/material cost.
Description: The swimmers consist of three or more segments ('***links***') and one less articulation joints ('***rotors***') - one rotor joint connects exactly two links to form a linear chain.
The swimmer is suspended in a two-dimensional pool and always starts in the same position (subject to some deviation drawn from a uniform distribution),
and the goal is to move as fast as possible towards the right by applying torque to the rotors and using fluid friction and you should write a reward function to make the robot move as faster as possible.

<mujoco model="swimmer">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option density="4000" integrator="RK4" timestep="0.01" viscosity="0.1"/>
  <default>
    <geom conaffinity="0" condim="1" contype="0" material="geom" rgba="0.8 0.6 .4 1"/>
    <joint armature='0.1'  />
  </default>
  <asset>
    <texture builtin="gradient" height="100" rgb1="1 1 1" rgb2="0 0 0" type="skybox" width="100"/>
    <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
    <texture builtin="checker" height="100" name="texplane" rgb1="0 0 0" rgb2="0.8 0.8 0.8" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.5" shininess="1" specular="1" texrepeat="30 30" texture="texplane"/>
    <material name="geom" texture="texgeom" texuniform="true"/>
  </asset>
  
  <worldbody>
    <light cutoff="100" diffuse="1 1 1" dir="-0 0 -1.3" directional="true" exponent="1" pos="0 0 1.3" specular=".1 .1 .1"/>
    <geom condim="3" material="MatPlane" name="floor" pos="0 0 -0.1" rgba="0.8 0.9 0.8 1" size="40 40 0.1" type="plane"/>
    <!--  ================= SWIMMER ================= /-->
    <body name="torso" pos="0 0 0">
      <camera name="track" mode="trackcom" pos="0 -3 3" xyaxes="1 0 0 0 1 1"/>
      <geom density="1000" fromto="{param1} 0 0 0 0 0" size="{param4}" type="capsule"/>
      <joint axis="1 0 0" name="slider1" pos="0 0 0" type="slide"/>
      <joint axis="0 1 0" name="slider2" pos="0 0 0" type="slide"/>
      <joint axis="0 0 1" name="free_body_rot" pos="0 0 0" type="hinge"/>
      <body name="mid" pos="0 0 0">
        <geom density="1000" fromto="0 0 0 -{param2} 0 0" size="{param5}" type="capsule"/>
        <joint axis="0 0 1" limited="true" name="motor1_rot" pos="0 0 0" range="-100 100" type="hinge"/>
        <body name="back" pos="-{param2} 0 0">
          <geom density="1000" fromto="0 0 0 -{param3} 0 0" size="{param6}" type="capsule"/>
          <joint axis="0 0 1" limited="true" name="motor2_rot" pos="0 0 0" range="-100 100" type="hinge"/>
        </body>
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor ctrllimited="true" ctrlrange="-1 1" gear="200.0" joint="motor1_rot"/>
    <motor ctrllimited="true" ctrlrange="-1 1" gear="200.0" joint="motor2_rot"/>
  </actuator>
</mujoco>

1. For reducing material cost to ensure the effieciency of robot design, you should reduce redundant paramters (e.g., smaller geom size) and increase paramters who control the robot (e.g., longer leags, ankles).
2. Your design should fit the control gear and others parts of robots well.

There are also some design parameters and their fitness. 
Please carefully observe these parameters and their fitness, try to design a new parameter to further improve the fitness.
"""

In [6]:
designer = DGA()
rewardfunc_list = [f'results/Eureka_morphology/env/GPTrewardfunc_{i}.py' for i in range(0,1)]
best_message = ""
for iter in range(iterations):
    morphology_list, material_list, parameter_list = designer.generate_morphology_eureka(morphology_nums, best_message, folder_name)
    for i, rewardfunc in enumerate(rewardfunc_list):
        for j, morphology in enumerate(morphology_list):

            print(i, rewardfunc)
            print(j, morphology)
            shutil.copy(morphology, "GPTSwimmer.xml")
            shutil.copy(rewardfunc, "GPTrewardfunc.py")         

            import GPTrewardfunc
            importlib.reload(GPTrewardfunc)  # 重新加载模块
            from GPTrewardfunc import _get_rew
            GPTSwimmerEnv._get_rew = _get_rew

            env_name = "GPTSwimmerEnv"
            model_path = Train(j,  i, folder_name, total_timesteps=5e5)
            fitness, reward = Eva(model_path)

            material = material_list[j]
            efficiency = fitness/material
            fitness_matrix[i][j] = fitness
            efficiency_matrix[i][j] = efficiency
            
            logging.info("___________________finish coarse optimization_____________________")
            logging.info(f"morphology: {j}, rewardfunc: {i}, material cost: {material} reward: {reward} fitness: {fitness} efficiency: {efficiency}")

    best_index = np.argmax(efficiency_matrix)
    best_efficiency = np.max(efficiency_matrix[:][0])
    best_parameter = parameter_list[best_index]
    best_message = eureka_morphology_prompts + f"\n best parameter:{best_parameter} \n" + f"best fintess:{best_efficiency}" 

Response 0:
"{\n  \"parameters\": [1.2, 1.0, 0.8, 0.05, 0.05, 0.05],\n  \"description\": \"Designed for optimal fluid locomotion, the swimmer model consists of three segments: a longer leading segment for initial propulsion followed by two slightly smaller segments. Reduced diameter segments promote lower material usage and resistance, enhancing speed and minimizing costs.\"\n}"
Response 1:
"{\n  \"parameters\": [1.0, 1.2, 1.3, 0.1, 0.1, 0.1],\n  \"description\": \"This swimmer design optimizes for fluid movement with elongated segments for efficient propulsion and minimal size for reduced material costs, enhancing swimming speed relative to build cost.\"\n}"
Response 2:
"{\n  \"parameters\": [1.0, 0.8, 0.6, 0.05, 0.04, 0.03],\n  \"desciption\": \"This design uses progressively shorter and thinner segments starting from the front to the back, optimizing for material efficiency while ensuring functional articulation for propulsion in a fluid environment.\"\n}"
Response 3:
"{\n  \"parame

ValueError: Expected parameter loc (Tensor of shape (1024, 2)) of distribution Normal(loc: torch.Size([1024, 2]), scale: torch.Size([1024, 2])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        ...,
        [nan, nan],
        [nan, nan],
        [nan, nan]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [1]:
best_efficiency

NameError: name 'best_efficiency' is not defined